In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt # for plotting the data 
import seaborn as sns # Advanced data plotting on top of matplotlib

import scipy.sparse as sp

In [2]:
#считывание данных


users = pd.read_csv('../input/tumenrecsys/train/users.csv', sep=';', index_col=None, dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})
items = pd.read_csv('../input/tumenrecsys/train/items.csv', sep=';', index_col=None, dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str})
train_transactions = pd.read_csv('../input/tumenrecsys/train/train_transactions_extended.csv', sep=';', index_col=None, dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str})

In [3]:
#train_transactions.sys_numb.unique().shape

In [4]:
users.head(10)

,chb,age,gender,chit_type
0,300001020830,21,female,нет данных
1,300001113642,36,female,нет данных
2,300001148466,46,female,нет данных
3,300001117011,22,female,нет данных
4,200001038094,24,female,echb
5,200001120504,22,female,нет данных
6,300000849123,27,male,нет данных
7,300001002599,21,female,нет данных
8,200000931895,28,female,нет данных
9,300000806816,23,female,normal


In [5]:
items.head(10)

,sys_numb,title,author,izd,year_izd,bbk
0,RSL01008600016,Судебное следствие в уголовном процессе России...,"Машовец, Асия Океановна",Юрлитинформ,2016,"Х629.374,0"
1,RSL01004304880,Уральское казачество и его роль в системе росс...,"Дубовиков, Александр Маратович",none,2006,none
2,RSL07000461043,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует
3,RSL07000433335,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует
4,RSL01002419013,"""Я пишу как эхо другого.."" : Очерки интеллекту...","Щедрина, Татьяна Геннадьевна",Прогресс-Традиция,2004,"Ю3(2)6-69Шпет Г.Г.-3,0"
5,RSL01004307424,Использование дробных физических нагрузок в ст...,"Статун, Алла Николаевна",none,2007,none
6,RSL07000467555,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует
7,RSL01009483558,Очерк психологии полиморфной индивидуальности ...,"Белоус, Валерий Владимирович",none,2016,none
8,RSL01004292734,"ФАКТОРЫ, ОПРЕДЕЛЯЮЩИЕ ПРИВЕРЖЕННОСТЬ К ЛЕЧЕНИЮ...","Бирюкова, Любовь Алексеевна",none,2009,none
9,RSL07000458490,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует


In [6]:
items.shape

(354355, 6)

In [7]:
#items.drop(items[items["title"] == "отсутствует"].index, inplace=True)

In [8]:
#т.к далее придётся работать с матрицами созданим словари, которые точно отображают индексы в chb/sys_numb и обратно

mapping_chb2index = {chb_number: index for index, chb_number in enumerate(users.chb.values)}
mapping_index2chb = {index: chb_number for index, chb_number in enumerate(users.chb.values)}

mapping_sys_numb2index = {sys_numb: index for index, sys_numb in enumerate(items.sys_numb.values)}
mapping_index2sys_numb= {index: sys_numb for index, sys_numb in enumerate(items.sys_numb.values)}


# NLP for title

In [9]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 642.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1333ffb0f52265c247681e13099bd16809cf84ea351838a859cd8fbaeabc4214
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [10]:
import torch

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
from sentence_transformers import SentenceTransformer

load_files = True

model = SentenceTransformer('sentence-transformers/LaBSE')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

In [13]:
model.cuda()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
)

In [14]:
title_vecs_w2v = model.encode(items.title)

Batches:   0%|          | 0/11074 [00:00<?, ?it/s]

**EDA**

In [15]:
nodata_books = set(items[items["title"] == "отсутствует"]["sys_numb"].values)


In [16]:
sample_bad = items[items["title"] == "отсутствует"]["sys_numb"].values[:10]
for item in sample_bad:
    print(train_transactions[train_transactions["sys_numb"] == item])
    

                 chb        sys_numb      date_1 is_real         type  \
257486  300001148466  RSL07000461043  2022-03-03     yes  книговыдача   

                     source is_printed  
257486  единый просмоторщик      False  
                 chb        sys_numb      date_1 is_real         type  \
250918  300001117011  RSL07000433335  2021-11-10     yes  книговыдача   

                     source is_printed  
250918  единый просмоторщик      False  
                 chb        sys_numb      date_1 is_real         type  \
162602  300000849123  RSL07000467555  2021-12-21     yes  книговыдача   

                     source is_printed  
162602  единый просмоторщик      False  
                 chb        sys_numb      date_1 is_real         type  \
154997  300000806816  RSL07000458490  2022-02-12     yes  книговыдача   

                     source is_printed  
154997  единый просмоторщик      False  
                 chb        sys_numb      date_1 is_real         type  \
136616  200

In [17]:
train_transactions.duplicated().sum()

0

In [18]:
for col in items.columns:
    print(col, len(items[col].unique()))

sys_numb 354355
title 319821
author 172622
izd 36909
year_izd 2506
bbk 130780


In [19]:
train_transactions.drop_duplicates(subset=["chb","sys_numb"], keep='first', inplace=True, ignore_index=False)

In [20]:
len(set(train_transactions.sys_numb).intersection(nodata_books))  

14796

In [21]:
print(train_transactions.groupby("chb")["sys_numb"].count().mean())
train_transactions.groupby("chb")["sys_numb"].count().sort_values()

13.669312958873038


chb
300000884658       2
100001111620       2
300000838729       2
300000838696       2
300000838230       2
                ... 
300000933517    1288
300001058749    1297
300000758346    1655
200000789223    3914
200000872413    6671
Name: sys_numb, Length: 16753, dtype: int64

In [22]:
#print(train_transactions.groupby("sys_numb")["chb"].count().mean())
#train_transactions.groupby("sys_numb")["chb"].count().sort_values()

In [23]:
#train_transactions.groupby("sys_numb")["chb"].count().sort_values().tail(30) 

In [24]:
#items[items["sys_numb"] == "RSL01009824096"]

In [25]:
#otkritki = items[items["title"] == "С Рождеством Христовым Изоматериал : Открытка"]["sys_numb"] 
#print(train_transactions.shape)
#for item in otkritki:
#    train_transactions.drop(train_transactions[train_transactions["sys_numb"]==item].index, inplace=True)
#print(train_transactions.shape)

In [26]:
#train_transactions.groupby("sys_numb")["chb"].count().sort_values().tail(10) 

In [27]:
#items[items["sys_numb"] == "RSL01009824143"]

In [28]:
#otkritki = items[items["title"] == "С Рождеством Христовым! Изоматериал : Открытка"]["sys_numb"] 
#print(train_transactions.shape)
#for item in otkritki:
#    train_transactions.drop(train_transactions[train_transactions["sys_numb"]==item].index, inplace=True)
#print(train_transactions.shape)

In [29]:
most_popular_index = train_transactions.groupby("sys_numb")["chb"].count().sort_values().tail(50).index 

In [30]:
for k in most_popular_index:
    print(items[items["sys_numb"] == k]["title"].values[0])

Церковная археология Москвы : храмы и приходы Ивановской горки и Кулишек
Всеобщая история интерьера 
Москва в кольце Садовых : путеводитель
История русского искусства : Русское искусство XX века
Архитектура советского авангарда : в 2 книгах / С. О. Хан-Магомедов : Социальные проблемы
С Рождеством Христовым! Изоматериал : Открытка
С Рождеством Христовым Изоматериал : Открытка
Зодчие Санкт-Петербурга, XVIII в. : Архитекторы барокко. Ранний классицизм. Строгий классицизм : Очерки
Мебель в русском интерьере конца XIX - начала XX веков
И по плодам узнается древо : рус. иконопись XV-XX веков из собр. Виктора Бондаренко
Русский стиль : Поиски выражения нац. самобытности. Народность и национальность. Традиции древнерус. и нар. искусства в рус. искусстве XVIII-нач. XX в.
С Рождеством Христовым Изоматериал : Открытка
Русские живописцы XVIII-XIX века : биографический словарь
С Рождеством Христовым! Изоматериал : Открытка
На рубеже веков… : искусство эпохи модерна
Теория литературы : Учеб. для сту

In [31]:
print(f"Кол-во пользователей: {len(train_transactions['chb'].unique())}")
print(f"Кол-во документов в истории пользователей: {len(train_transactions['sys_numb'].unique())}")
print(f"Общее кол-во документов: {len(items['sys_numb'].unique())}")

Кол-во пользователей: 16753
Кол-во документов в истории пользователей: 194666
Общее кол-во документов: 354355


In [32]:
#в базовом решении будем не будем использовать дополнительные данные о взаимодействиях
train_transactions = train_transactions[['chb', 'sys_numb']]
train_transactions.head()

,chb,sys_numb
0,100000641403,RSL01004206702
1,100000641403,RSL01000769304
2,100000641403,RSL01004211574
3,100000644359,RSL01009800093
4,100000644359,RSL01003557352


In [33]:
#т.к далее придётся работать с матрицами созданим словари, которые точно отображают индексы в chb/sys_numb и обратно

mapping_chb2index = {chb_number: index for index, chb_number in enumerate(users.chb.values)}
mapping_index2chb = {index: chb_number for index, chb_number in enumerate(users.chb.values)}

mapping_sys_numb2index = {sys_numb: index for index, sys_numb in enumerate(items.sys_numb.values)}
mapping_index2sys_numb= {index: sys_numb for index, sys_numb in enumerate(items.sys_numb.values)}

In [34]:
train_transactions["chb_i"] = train_transactions["chb"].apply(lambda a: mapping_chb2index[a])
train_transactions["sys_num_i"] = train_transactions["sys_numb"].apply(lambda a: mapping_sys_numb2index[a])

In [35]:
train_coo = sp.coo_matrix(
    (np.ones(len(train_transactions)), (train_transactions.chb_i, train_transactions.sys_num_i)),
    shape=(len(mapping_chb2index), len(mapping_sys_numb2index))
)
train_coo

<16753x354355 sparse matrix of type '<class 'numpy.float64'>'
	with 229002 stored elements in COOrdinate format>

# NLP Prediction

In [36]:
def top_n_nearesst_index(embeddings, item_to_predict = 0, n = 5): 

    cs_list = []
    #print(embeddings[item_to_predict].reshape(1, -1))
    
    for item in embeddings:
        #cs = cosine_similarity(item.reshape(1, -1), embeddings[item_to_predict].reshape(1, -1))
        
        # np cosine_similarity works faster
        corrcoef = np.correlate(item, embeddings[item_to_predict])

        cs_list.append(corrcoef[0])
    
    
    argsort_scores_idx = np.argsort(cs_list)[-n-1:-1]
    argsort_scores_idx = np.flip(argsort_scores_idx)

    return argsort_scores_idx

In [37]:
import torch
import torch.nn as nn
import numpy as np

cos = nn.CosineSimilarity(dim=1, eps=1e-6)

def top_n_nearesst_index_type2(embeddings, item_to_predict = 0, n = 5): 

    cs_list = []
    #print(embeddings[item_to_predict].reshape(1, -1))
    
    for item in embeddings:
        #cs = cosine_similarity(item.reshape(1, -1), embeddings[item_to_predict].reshape(1, -1))
        
        # np cosine_similarity works faster
        corrcoef = cos(item, embeddings[item_to_predict])

        cs_list.append(corrcoef[0])
    
    
    argsort_scores_idx = np.argsort(cs_list)[-n-1:-1]
    argsort_scores_idx = np.flip(argsort_scores_idx)

    return argsort_scores_idx

In [38]:
most_popular = [mapping_sys_numb2index[item] for item in most_popular_index[:20]]

In [39]:
all_rec = []

embeddings = title_vecs_w2v

# go through all customers from test list
for customer in users.chb.values[:5]:
    
    
    # go through all purchases of customers
    customer_pred = []
    
    print(customer)
    print("*"*40)
    
    for k in train_transactions[train_transactions.chb == customer].sys_numb.values:
        print(items[items["sys_numb"] == k]["title"].values[0])
    
    print("*"*40)
    
    num = len(train_transactions[train_transactions.chb == customer].sys_numb)
    
    #samp_n = max(20%num, 1)
    
    for sys_numb in train_transactions[train_transactions.chb == customer].sys_numb:
        
        if sys_numb in nodata_books: continue
            
        article_index = mapping_sys_numb2index[sys_numb]
        
        pred = top_n_nearesst_index(embeddings, article_index, 20)
         
        customer_pred += list(pred)
        
    
    if len(customer_pred) == 0:
        customer_pred = most_popular
    else:
        customer_pred = np.unique(customer_pred)[:20]
    
    
    for k in customer_pred:
        book = mapping_index2sys_numb[k]
        all_rec.append([customer, book])
        
        print(items[items["sys_numb"] == book]["title"].values[0])
        
    #full_pred_list.append(np.unique(customer_pred))
    print()
    print()
    print()

300001020830
****************************************
Европейские стандарты справедливого судебного разбирательства по уголовным делам : монография
Уголовная политика Европейского союза в сфере противодействия коррупции : монография
отсутствует
отсутствует
Проблемы реализации принципов криминализации и положений законодательной техники применительно к нормам главы 22 УК РФ : диссертация .. кандидата юридических наук : 12.00.08
Основы законодательной техники в уголовном праве России : теория и законодательная практика : диссертация .. доктора юридических наук : 12.00.08
Криминализация (декриминализация) в уголовном праве России : диссертация .. кандидата юридических наук : 12.00.08
Законодательная техника в уголовном праве : автореферат дис.. кандидата юридических наук : 12.00.08
Преступления против собственности : учебное пособие
Механизм формирования результатов «невербальных» следственных и судебных действий в уголовном судопроизводстве 
Преступный монополизм: уголовно-политическое и

In [40]:
import time

In [41]:
all_rec = []

embeddings = title_vecs_w2v

all_len = len(users.chb.values)
ct = time.time()
# go through all customers from test list
for ii, customer in enumerate(users.chb.values):
    if ii % 100 == 0:
        print(ii, "/", 16753)# all_len)
    
        print(time.time() - ct)
        ct = time.time()
    
    customer_pred = []
    
   
    num = len(train_transactions[train_transactions.chb == customer].sys_numb)
    
    st = 0
    
    for sys_numb in train_transactions[train_transactions.chb == customer].sys_numb:
        
        if sys_numb in nodata_books: continue
        
        st += 1
        if st == 3: break
            
        article_index = mapping_sys_numb2index[sys_numb]
        
        pred = top_n_nearesst_index(embeddings, article_index, 20)
         
        customer_pred += list(pred)
        
    
    if len(customer_pred) == 0:
        customer_pred = most_popular
    else:
        customer_pred = np.unique(customer_pred)[:20]
        
    for k in customer_pred:
        book = mapping_index2sys_numb[k]
        all_rec.append([customer, book])
        
        
all_rec[:3]

0 / 16753
0.0006086826324462891
100 / 16753
203.59179663658142
200 / 16753
206.07757449150085
300 / 16753
206.43415141105652
400 / 16753
205.5844178199768
500 / 16753
205.93411827087402
600 / 16753
205.8212080001831
700 / 16753
205.47819709777832
800 / 16753
200.97196459770203
900 / 16753
205.48366975784302
1000 / 16753
202.78214931488037
1100 / 16753
206.99643564224243
1200 / 16753
203.78571105003357
1300 / 16753
201.06598162651062
1400 / 16753
195.88571858406067
1500 / 16753
199.94420289993286
1600 / 16753
203.0315272808075
1700 / 16753
205.20114374160767
1800 / 16753
200.62624669075012
1900 / 16753
201.3140573501587
2000 / 16753
205.93546175956726
2100 / 16753
204.07139039039612
2200 / 16753
203.5706729888916
2300 / 16753
202.79496932029724
2400 / 16753
208.22329354286194
2500 / 16753
205.38204956054688
2600 / 16753
205.26663494110107
2700 / 16753
206.95222854614258
2800 / 16753
206.6262595653534
2900 / 16753
204.3557641506195
3000 / 16753
205.718976020813
3100 / 16753
205.658375501

[['300001020830', 'RSL01008029976'],
 ['300001020830', 'RSL01008061093'],
 ['300001020830', 'RSL01008924631']]

In [42]:
# DataFrame для отправки решения с рекомендациями
solution = pd.DataFrame(all_rec, columns=["chb", "sys_numb"])
print(solution.chb.unique().shape)
print(solution.sys_numb.unique().shape)
solution.to_csv("solution_02.csv", index=False, sep=';')

(16753,)
(132180,)
